## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/patData.txt"
file_type = "txt"
data = sc.textFile(file_location)
dataInfo, scoreInfo = data.collect()[:2]
maxNum, typeNum, dataNum = [int(i) for i in dataInfo.split(' ')]
typeNum = sc.broadcast(typeNum)
print('user_id problem_id partial_score_obtained')
data.collect()

user_id problem_id partial_score_obtained
 Out[ 28 ]: ['7 4 20',
 '20 25 25 30',
 '00002 2 12',
 '00007 4 17',
 '00005 1 19',
 '00007 2 25',
 '00005 1 20',
 '00002 2 2',
 '00005 1 15',
 '00001 1 18',
 '00004 3 25',
 '00002 2 25',
 '00005 3 22',
 '00006 4 -1',
 '00001 2 18',
 '00002 1 20',
 '00004 1 15',
 '00002 4 18',
 '00001 3 4',
 '00001 4 2',
 '00005 2 -1',
 '00004 2 0']

In [3]:
# 获取行号
# 过滤开头两行
# 划分数据”ID Problem“,“Score”
# 获取多次提交的最佳成绩
# 划分数据“ID”,“ID Problem Score”
# 把每个人的数据合并
# 根据Key排序
scoreData = data.zipWithIndex() \
  .filter(lambda x: x[1]>1) \
  .map(lambda x:(" ".join(x[0].split(" ")[:2]), x[0].split(" "))) \
  .reduceByKey(lambda x,y: x[2] >= y[2] and x or y) \
  .map(lambda x: (x[1][0], x[1])) \
  .reduceByKey(lambda x,y: [x[0], ','.join([x[1], y[1]]), ','.join([x[2], y[2]])]) \
  .sortByKey()
scoreData.collect()

Out[ 45 ]: [('00001', ['00001', '4,2,1,3', '2,18,18,4']),
 ('00002', ['00002', '4,2,1', '18,25,20']),
 ('00004', ['00004', '2,1,3', '0,15,25']),
 ('00005', ['00005', '2,3,1', '-1,22,20']),
 ('00006', ['00006', '4', '-1']),
 ('00007', ['00007', '2,4', '25,17'])]

In [4]:
# 填补没有提交的问题分数为-，并计算总分，最后key为“成功解题个数”、“ID”，“总分数”，“分数”
# 排序，ID_asc-成功解题个数_asc-总分数_desc————意为成功解题少但总分又多的
def sumColScore(x):
  scoreList = ['-' for i in range(typeNum.value)]
  sumScore = 0
  typeId = x[1].split(',')
  scoreTmp = x[2].split(',')
  cnt = 0
  for k,v in enumerate(typeId):
    if scoreTmp[k] == '-1': continue
    cnt += 1
    scoreList[int(v)-1] = scoreTmp[k]
    sumScore += int(scoreTmp[k])
  return [str(cnt), x[0], str(sumScore), ' '.join(scoreList)]
sumData = scoreData.map(lambda x: (x[0], sumColScore(x[1]))) \
  .sortBy(lambda x:x[1][1], ascending=True) \
  .sortBy(lambda x:x[1][0], ascending=True) \
  .sortBy(lambda x:x[1][2], ascending=False) 
sumData.collect()

Out[ 58 ]: [('00002', ['3', '00002', '63', '20 25 - 18']),
 ('00005', ['2', '00005', '42', '20 - 22 -']),
 ('00007', ['2', '00007', '42', '- 25 - 17']),
 ('00001', ['4', '00001', '42', '18 18 4 2']),
 ('00004', ['3', '00004', '40', '15 0 25 -']),
 ('00006', ['0', '00006', '0', '- - - -'])]

In [5]:
# 获取行号
# 分割数据“行号”，“ID SumScore Scores”
# 根据分数聚合
# 根据分数降序排序
rankData = sumData.zipWithIndex() \
  .map(lambda x: (x[1], x[0][1])) \
  .groupBy(lambda x: x[1][2]) \
  .sortByKey(ascending= False) \
  .collect()
flag = "-"
rem_index=-1
for index, t in enumerate(rankData):
    for tt in t[1]:
        if flag == tt[1][2]:
            print("{} {}".format(rem_index, " ".join(tt[1][1:])))
        else:
            flag = tt[1][2]
            rem_index = tt[0]+1
            print("{} {}".format(rem_index, " ".join(tt[1][1:])))

1 00002 63 20 25 - 18
2 00005 42 20 - 22 -
2 00007 42 - 25 - 17
2 00001 42 18 18 4 2
5 00004 40 15 0 25 -
6 00006 0 - - - -